Martín Amado - 19020
Juan Pablo Pineda - 19087

referencia para análisis de datos: https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python#4.-Missing-data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.cm as cm
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from apyori import apriori

## Análisis  exploratorio
Explicación de hallazgos y conclusiones

In [ ]:
data = pd.read_csv('./train.csv', encoding = "latin1")
file = open('cuantitativas.txt', 'r')
quant= file.read().splitlines()
file = open('cualitativas.txt', 'r')
quali= file.read().splitlines()

## Descripcion de los datos

In [ ]:
#Variables cuantitativas
data[quant].describe()

## Analisis de los datos

### Correlacion en variables cuantitativas

In [ ]:
datadataddadatak = 11
corrmat = data.corr()
columns = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(data[columns].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=columns.values, xticklabels=columns.values)
plt.show()

### Kurtosis, Asimetria y Gráfico de las 11 variables con mayor correlación

In [ ]:
for var in columns:
    localData = data[var].dropna(how='all', axis=0)
  
    sns.displot(localData, kde=True)

    print( var, ':', ' Kurtosis:', stats.kurtosis(localData), 'Asimetría:', stats.skew(localData), '\n')

### Gráfico de pares de las 11 variables cuantitativas con mayor correlación

In [ ]:
sns.set()
sns.pairplot(data[columns], height= 5)
plt.show()

### Gráficos de barras de las variables cualitativas

In [ ]:
#Variables cualitativas
pd.set_option("display.max_rows", data.shape[0]+1)
for var in quali:
  #Se omite ID ya que, en este caso no aporta ninguna informacion importante.
  if(var == "Id"):
    pass
  #Se reemplaza el grafico de barras en estas variables por una tabla de frecuencias ya que la informacion era ilegible
  elif(var in ["Neighborhood", "YearBuilt", "YearRemodAdd", "GarageYrBlt"]):
    print(data[var].value_counts(), "\n")
  #muestra graficos de barras legibles.
  else:
    data[var].value_counts().plot(kind='barh')
    plt.figure(figsize=(20,5))
    print(var)
    plt.show()

### Gráficos de Caja y Bigotes de las variables Cuantitativas

In [ ]:
data.boxplot(column=["OverallQual", "GarageArea", "YearRemodAdd"])

### Kurtosis, Asimetria y Gráfico de las otras variables cuantitativas

In [ ]:
#Variables cualitativas
for var in quali:
  data[var].value_counts().plot(kind='barh')
  plt.figure(figsize=(20,5))
  print(var)
  plt.show()

### Kurtosis, Asimetria y Gráfico de las otras variables cuantitativas

In [ ]:
for var in quant:
  if var not in columns:
    localData = data[var].dropna(how='all', axis=0)
  
    sns.displot(localData, kde=True)

    print( var, ':', ' Kurtosis:', stats.kurtosis(localData), 'Asimetría:', stats.skew(localData), '\n')

### Análisis PCA

De acuerdo con la matriz de correlación presentada anteriormente, se observan 3 fenómenos importantes:
-Dentro de las 11 variables con mayor correlación, hay correlación alta y baja
-La correlación alta por lo general es entre variables que se conectan por lo que miden (garage con cantidad de carros)
-La correlación baja se da entre variables de tamaño vs año.

####  Indice  KMO  y  el  test  de  esfericidad  de  Bartlett

In [ ]:
### LotFrontage Nan is replaced with 0, assuming no length of street touches the property
data['LotFrontage'] = data['LotFrontage'].fillna(0)

### MasVnrArea Nan is replaced with 0, assuming no Veneer is present therefor the area is 0
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)

In [ ]:
chi_cuadrado, p_valor = calculate_bartlett_sphericity(data[quant])
chi_cuadrado, p_valor

Analisis: El valor de p es menor a 0.05, lo que indica correlacion entre variables y el rechazo de la hipotesis nula

In [ ]:
kmo,kmo_modelo = calculate_kmo(data[quant])
kmo_modelo

Analisis: Con un indice mayor a 0.5, la adecuacion muestral para analisis factorial es valida, y se puede hacer el analisis de componentes principales

### Analisis de Componentes Principales

In [ ]:
pca_pipe = make_pipeline(StandardScaler(), PCA())
pca_pipe.fit(data[quant])

pca_model = pca_pipe.named_steps['pca']

pd.DataFrame(
  data = pca_model.components_,
  columns = data[quant].columns,
  index = ['PC1','PC2','PC3','PC5','PC6','PC7','PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30']
)

#### Variable y su componente

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
componentes = pca_model.components_
plt.imshow(componentes.T)
plt.yticks(range(len(data[quant].columns)), data[quant].columns)
plt.xticks(range(len(data[quant].columns)), np.arange(pca_model.n_components_)+1)
plt.grid(False)
plt.colorbar();
fig.set_size_inches(12, 12)

### Varianza por componente

In [ ]:
pca_model.explained_variance_ratio_

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax.bar(
    x      = np.arange(pca_model.n_components_) + 1,
    height = pca_model.explained_variance_ratio_
)

for x, y in zip(np.arange(len(data[quant].columns)) + 1, pca_model.explained_variance_ratio_):
    label = round(y, 2)
    ax.annotate(
        label,
        (x,y),
        textcoords="offset points",
        xytext=(0,10),
        ha='center'
    )

ax.set_xticks(np.arange(pca_model.n_components_) + 1)
ax.set_ylim(0, 1.1)
ax.set_title('Porcentaje de varianza explicada por cada componente')
ax.set_xlabel('Componente principal')
ax.set_ylabel('Por. varianza explicada')

In [ ]:
prop_varianza_acum = pca_model.explained_variance_ratio_.cumsum()
print('------------------------------------------')
print('Porcentaje de varianza explicada acumulada')
print('------------------------------------------')
print(prop_varianza_acum)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
ax.plot(
    np.arange(len(data[quant].columns)) + 1,
    prop_varianza_acum,
    marker = 'o'
)

for x, y in zip(np.arange(len(data[quant].columns)) + 1, prop_varianza_acum):
    label = round(y, 2)
    ax.annotate(
        label,
        (x,y),
        textcoords="offset points",
        xytext=(0,10),
        ha='center'
    )
    
ax.set_ylim(0, 1.1)
ax.set_xticks(np.arange(pca_model.n_components_) + 1)
ax.set_title('Porcentaje de varianza explicada acumulada')
ax.set_xlabel('Componente principal')
ax.set_ylabel('Por. varianza acumulada')

Para mantener el mayor porcentaje de la varianza posible, se tomaron 21 dimensiones. Esto permite tener hasta 95.60% de la varianza.

### Reglas de asociacion apriori

### Conclusiones

Una gran parte de los datos tienen una distribución normal, lo que permite hacer asunciones para guiar el proceso del análisis.

Los componentes principales nos dan a entender dos cosas principales

1. Las variables que representan 'lujos' suelen tener menor correlacion, pues aumentan el precio de una casa, pero no son la unica dependencia.

2. La mayor correlacion y agrupaciones se dan en las variables como: Tamaños de patio, si existe dicho patio. Tamaño de garage, si existe un garage.